## Install SQLite

In [1]:
!pip install ipython-sql

In [2]:
%reload_ext sql
%sql sqlite://

---

## Create datasets

In [3]:
%%sql

CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');
 

CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');
  

CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

 * sqlite://
Done.
15 rows affected.
Done.
3 rows affected.
Done.
2 rows affected.


[]

#### Table 1: sales

In [4]:
%%sql
select * from sales;

 * sqlite://
Done.


customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2
A,2021-01-10,3
A,2021-01-11,3
A,2021-01-11,3
B,2021-01-01,2
B,2021-01-02,2
B,2021-01-04,1
B,2021-01-11,1


#### Table 2: menu

In [5]:
%%sql
select * from menu;

 * sqlite://
Done.


product_id,product_name,price
1,sushi,10
2,curry,15
3,ramen,12


#### Table 3: members

In [6]:
%%sql
select * from members;

 * sqlite://
Done.


customer_id,join_date
A,2021-01-07
B,2021-01-09


---

## Case Study Questions & My Solutions

### 1. What is the total amount each customer spent at the restaurant?

In [7]:
%%sql

DROP TABLE IF EXISTS joined_table; 
CREATE TEMP TABLE joined_table AS

SELECT
  sales.customer_id,
  sales.order_date,
  sales.product_id,
  menu.product_name,
  menu.price,
  members.join_date
FROM sales
INNER JOIN menu
  ON menu.product_id = sales.product_id
LEFT JOIN members
  ON members.customer_id = sales.customer_id;

select * from joined_table;

 * sqlite://
Done.
Done.
Done.


customer_id,order_date,product_id,product_name,price,join_date
A,2021-01-01,1,sushi,10,2021-01-07
A,2021-01-01,2,curry,15,2021-01-07
A,2021-01-07,2,curry,15,2021-01-07
A,2021-01-10,3,ramen,12,2021-01-07
A,2021-01-11,3,ramen,12,2021-01-07
A,2021-01-11,3,ramen,12,2021-01-07
B,2021-01-01,2,curry,15,2021-01-09
B,2021-01-02,2,curry,15,2021-01-09
B,2021-01-04,1,sushi,10,2021-01-09
B,2021-01-11,1,sushi,10,2021-01-09


In [8]:
%%sql

SELECT
  customer_id,
  SUM(price) AS amount_spent
FROM joined_table
GROUP BY customer_id
ORDER BY customer_id;

 * sqlite://
Done.


customer_id,amount_spent
A,76
B,74
C,36


**Ans:** Customer A, B and C spent \\$76, \\$74 and \\$36 respectively. 

---

### 2. How many days has each customer visited the restaurant?

In [9]:
%%sql

SELECT
  customer_id,
  COUNT(DISTINCT order_date) AS days_visited
FROM joined_table
GROUP BY customer_id;

 * sqlite://
Done.


customer_id,days_visited
A,4
B,6
C,2


**Ans:** Customer A, B and C visited the restraurant 4 days, 6 days and 2 days respectively.

---

### 3. What was the first item from the menu purchased by each customer?

In [10]:
%%sql 

WITH orderRank AS (
    SELECT
        customer_id,
        order_date,
        product_name,
        DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY order_date) AS _rank
    FROM joined_table
)

SELECT  
    customer_id,
    product_name
FROM orderRank
WHERE _rank=1
GROUP BY customer_id,
         product_name;

 * sqlite://
Done.


customer_id,product_name
A,curry
A,sushi
B,curry
C,ramen


**Ans:** Customer A's first purchase was curry and sushi, Customer B's first purchase was curry while Custoemr C's first purchase was ramen.

---

### 4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [11]:
%%sql

SELECT 
  product_name,
  COUNT(product_name) AS times_purchased
FROM joined_table
GROUP BY product_name
ORDER BY times_purchased DESC;

 * sqlite://
Done.


product_name,times_purchased
ramen,8
curry,4
sushi,3


**Ans:** The most purchased item is ramen and it was purchased 8 times in total by all customers.

---

### 5. Which item was the most popular for each customer?

In [12]:
%%sql

WITH cte_most_popular_items AS(
SELECT
  customer_id,
  product_name,
  COUNT(*) AS times_purchased,
  DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY COUNT(*) DESC) AS _rank_item
FROM joined_table
GROUP BY
  customer_id,
  product_name
)

SELECT 
  customer_id,
  product_name,
  times_purchased
FROM cte_most_popular_items
WHERE _rank_item = 1;

 * sqlite://
Done.


customer_id,product_name,times_purchased
A,ramen,3
B,sushi,2
B,ramen,2
B,curry,2
C,ramen,3


**Ans:** The most popular item for Customer A is ramen and Customer B is ramen. Customer B has no favourite hence no 'most popular' item.

---

### 6. Which item was purchased first by the customer after they became a member?

In [13]:
%%sql

WITH cte_after_join AS(
SELECT *,
  DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY order_date) AS ranked_item_after_join
FROM joined_table
WHERE order_date >= join_date
GROUP BY customer_id    
)

SELECT
  customer_id,
  order_date,
  product_name
FROM cte_after_join
WHERE ranked_item_after_join = 1;

 * sqlite://
Done.


customer_id,order_date,product_name
A,2021-01-07,curry
B,2021-01-11,sushi


**Ans:** The first item purchased by Customer A after becoming a member is curry while Customer B is sushi. Note, Customer B did not join the membership.

---

### 7. Which item was purchased just before the customer became a member?

In [14]:
%%sql

WITH cte_before_join AS(
SELECT *,
  DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY order_date DESC) AS ranked_item_after_join
FROM joined_table
WHERE order_date < join_date
)

SELECT 
  customer_id,
  order_date,
  product_name
FROM cte_before_join
WHERE ranked_item_after_join = 1;

 * sqlite://
Done.


customer_id,order_date,product_name
A,2021-01-01,sushi
A,2021-01-01,curry
B,2021-01-04,sushi


**Ans:** The item purchased before Customer A became a member was sushi and curry while Customer B is sushi.

---

### 8. What is the total items and amount spent for each member before they became a member?

In [15]:
%%sql

SELECT
    customer_id,
    COUNT(product_id) AS total_unique_items,
    SUM(price) AS total_amount_spent
FROM joined_table
WHERE order_date < join_date
GROUP BY customer_id;

 * sqlite://
Done.


customer_id,total_unique_items,total_amount_spent
A,2,25
B,3,40


**Ans:** After becoming a member, Customer A purchased 2 items and spent \\$25 while Customer B purchased 3 items and spent \\$40.

---

### 9. If each \\$1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [16]:
%%sql

SELECT 
    customer_id,
    SUM(CASE
       WHEN product_name = 'sushi' THEN 2*10*price
       ELSE 10*price END) AS total_points
FROM joined_table
WHERE order_date >= join_date
GROUP BY customer_id
ORDER BY customer_id;

 * sqlite://
Done.


customer_id,total_points
A,510
B,440


**Ans:** Customer A would have 510 pts while Customer B would have 440 pts.

---

### 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [17]:
%%sql

SELECT 
    customer_id,
    SUM(CASE
       WHEN product_name = 'sushi' THEN 2*10*price
       WHEN order_date - join_date <= 6 THEN 2*10*price
       ELSE 10*price
       END) AS total_points
FROM joined_table
WHERE order_date >= join_date AND order_date < '2021-02-01'
GROUP BY customer_id
ORDER BY customer_id;

 * sqlite://
Done.


customer_id,total_points
A,1020
B,440


**Ans:** Customer A has 1020 pts while Customer B has 440 pts at the end of January.

---

### Bonus Questions

### 1. Join All The Things 

In [18]:
%%sql

DROP TABLE IF EXISTS all_joins;
CREATE TEMP TABLE all_joins AS

SELECT
  sales.customer_id,
  sales.order_date,
  menu.product_name,
  menu.price,
    CASE
    WHEN order_date >= join_date THEN 'Y'
    ELSE 'N' END AS member
FROM sales
INNER JOIN menu
  ON menu.product_id = sales.product_id
LEFT JOIN members
  ON members.customer_id = sales.customer_id;
    
select * from all_joins;

 * sqlite://
Done.
Done.
Done.


customer_id,order_date,product_name,price,member
A,2021-01-01,sushi,10,N
A,2021-01-01,curry,15,N
A,2021-01-07,curry,15,Y
A,2021-01-10,ramen,12,Y
A,2021-01-11,ramen,12,Y
A,2021-01-11,ramen,12,Y
B,2021-01-01,curry,15,N
B,2021-01-02,curry,15,N
B,2021-01-04,sushi,10,N
B,2021-01-11,sushi,10,Y


### 2. Rank All The Things

In [19]:
%%sql

WITH data_table AS (
SELECT
  sales.customer_id,
  sales.order_date,
  menu.product_name,
  menu.price,
    CASE
    WHEN order_date >= join_date THEN 'Y'
    ELSE 'N' END AS member
FROM sales
INNER JOIN menu
  ON menu.product_id = sales.product_id
LEFT JOIN members
  ON members.customer_id = sales.customer_id)

SELECT *,
  CASE WHEN member = 'Y' 
  	THEN DENSE_RANK() OVER(PARTITION BY customer_id, member ORDER BY order_date)
  ELSE 'null' END AS ranking
FROM data_table;

 * sqlite://
Done.


customer_id,order_date,product_name,price,member,ranking
A,2021-01-01,sushi,10,N,null
A,2021-01-01,curry,15,N,null
A,2021-01-07,curry,15,Y,1
A,2021-01-10,ramen,12,Y,2
A,2021-01-11,ramen,12,Y,3
A,2021-01-11,ramen,12,Y,3
B,2021-01-01,curry,15,N,null
B,2021-01-02,curry,15,N,null
B,2021-01-04,sushi,10,N,null
B,2021-01-11,sushi,10,Y,1
